<!-- Projeto Desenvolvido por Clênio Moura -->
# <font color='blue'>Analista de dados</font>
# <font color='blue'>Projetos de Análise de Dados com Linguagem Python</font>
# <font color='blue'>Projeto 1 - Extraindo e Analisando Dados de Bancos de Dados</font>

## Pacotes Python Usados no Projeto

In [1]:
# Instala o pacote watermark
!pip install -q -U watermark

In [2]:
# Instala o pacote iPython-sql 
!pip install -q ipython-sql

In [3]:
!pip install -q memory_profiler

In [4]:
# Imports
import os
import pandas as pd
import sqlite3
from memory_profiler import profile

https://www.sqlite.org/about.html

In [5]:
%reload_ext watermark
%watermark -a "Clênio Moura"

Author: Clênio Moura



## Criando o Banco de Dados Relacional

In [6]:
# Cria um dataframe com os dados de origem
dados = pd.DataFrame({'nivel': ['Junior', 'Pleno', 'Senior'],
                      'salario': [7500, 14650, 18320],
                      'cargo': ['Analista de Dados', 'Cientista de Dados', 'Engenheiro de Dados']})

In [7]:
# Lista os dados
dados.head()

,nivel,salario,cargo
0,Junior,7500,Analista de Dados
1,Pleno,14650,Cientista de Dados
2,Senior,18320,Engenheiro de Dados


In [8]:
# Define o caminho para o arquivo do banco de dados
arquivo_path = 'dsadatabase.db'

In [9]:
# Verifica se o arquivo existe e deleta se existir para criar um novo arquivo depois
if os.path.exists(arquivo_path):
    try:
        os.remove(arquivo_path)
        print(f"Arquivo {arquivo_path} deletado com sucesso!")
    except Exception as e:
        print(f"Erro ao deletar o arquivo {arquivo_path}. Detalhes: {e}")
else:
    print(f"Arquivo {arquivo_path} não encontrado.")

Arquivo dsadatabase.db deletado com sucesso!


## Conectando ao Banco de Dados com Linguagem Python

In [10]:
# Criamos a conexão a um banco de dados SQLite
cnn = sqlite3.connect('dsadatabase.db')

In [11]:
# Copia o dataframe para dentro do banco de dados como uma tabela
dados.to_sql('funcionarios', cnn)

3

In [12]:
# Carregamos a extensão SQL
%load_ext sql

In [13]:
# Definimos o banco de dados
%sql sqlite:///dsadatabase.db

> Agora executamos nossas consultas SQL usando diretamente Linguagem SQL dentro do Jupyter Notebook.

In [14]:
%%sql

SELECT * FROM funcionarios

 * sqlite:///dsadatabase.db
Done.


index,nivel,salario,cargo
0,Junior,7500,Analista de Dados
1,Pleno,14650,Cientista de Dados
2,Senior,18320,Engenheiro de Dados


In [15]:
%%sql

SELECT count(*) FROM funcionarios

 * sqlite:///dsadatabase.db
Done.


count(*)
3


In [16]:
%%sql

SELECT round(avg(salario),2) as 'salario_medio' FROM funcionarios

 * sqlite:///dsadatabase.db
Done.


salario_medio
13490.0


## Carregando o Banco de Dados a Partir de Arquivos CSV

Temos em mãos um arquivo com dados de pacientes que desenvolveram ou não diabetes. Vamos colocar o conteúdo do arquivo em um banco de dados.

In [17]:
# Carrega o dataset
df = pd.read_csv('dataset.csv')

In [18]:
type(df)

pandas.core.frame.DataFrame

In [19]:
df.shape

(768, 9)

In [20]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,103,30,38,83,43.3,0.183,33,0
1,1,115,70,30,96,34.6,0.529,32,1
2,3,126,88,41,235,39.3,0.704,27,0
3,8,99,84,0,0,35.4,0.388,50,0
4,7,196,90,0,0,39.8,0.451,41,1


In [21]:
# Copia o dataframe para dentro do banco de dados como uma tabela
df.to_sql('diabetes', cnn)

768

In [22]:
%%sql

SELECT count(*) FROM diabetes

 * sqlite:///dsadatabase.db
Done.


count(*)
768


## Sintaxe SQL e Sintaxe Pandas

> Retorne os pacientes que têm BMI maior que 52 e idade (Age) entre 25 e 30 anos.

**Sintaxe de consulta do Pandas:**

In [23]:

df.query("BMI > 52 and 25 <= Age <= 30")

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
120,0,162,76,56,100,53.2,0.759,25,1
125,1,88,30,42,99,55.0,0.496,26,1
177,0,129,110,46,130,67.1,0.319,26,1
303,5,115,98,0,0,52.9,0.209,28,1
445,0,180,78,63,14,59.4,2.420,25,1


**Sintaxe de consulta com SQL:**

In [24]:
%%sql

SELECT * FROM diabetes WHERE BMI > 52 AND Age BETWEEN 25 AND 30

 * sqlite:///dsadatabase.db
Done.


index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
120,0,162,76,56,100,53.2,0.759,25,1
125,1,88,30,42,99,55.0,0.496,26,1
177,0,129,110,46,130,67.1,0.319,26,1
303,5,115,98,0,0,52.9,0.209,28,1
445,0,180,78,63,14,59.4,2.42,25,1


## Respondendo Perguntas de Negócio com Análise de Dados

Precisamos gerar uma amostra de dados com os pacientes com mais de 50 anos e para cada um deles indicar em uma nova coluna se o paciente está normal (BMI menor que 30) ou obeso (BMI maior ou igual a 30). Então devemos gerar um novo arquivo CSV e encaminhar para o tomador de decisão.

**Retorne Age, Glucose e Outcome para pacientes com Glucose maior que 195.**

In [25]:
%%sql

SELECT Age, Glucose, Outcome FROM diabetes WHERE Glucose > 195

 * sqlite:///dsadatabase.db
Done.


Age,Glucose,Outcome
41,196,1
53,197,1
57,196,1
31,197,0
29,196,1
39,197,1
28,198,1
62,197,1
22,199,1


In [26]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

**Vamos criar uma nova tabela no banco de dados.**

In [27]:
%%sql

CREATE TABLE pacientes (Pregnancies INT,
                        Glucose INT,
                        BloodPressure INT,
                        SkinThickness INT,
                        Insulin INT,
                        BMI DECIMAL(8, 2),
                        DiabetesPedigreeFunction DECIMAL(8, 2),
                        Age INT,
                        Outcome INT);

 * sqlite:///dsadatabase.db
Done.


[]

**A tabela está vazia.**

In [28]:
%%sql

SELECT * FROM pacientes

 * sqlite:///dsadatabase.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome


**Vamos copiar o conteúdo de uma tabela para outra, mas somente para pacientes maiores que 50 anos de idade.**

In [29]:
%%sql

INSERT INTO pacientes(Pregnancies, 
                      Glucose, 
                      BloodPressure, 
                      SkinThickness, 
                      Insulin, 
                      BMI, 
                      DiabetesPedigreeFunction, 
                      Age, 
                      Outcome) 
SELECT Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age, Outcome 
FROM diabetes WHERE Age > 50;

 * sqlite:///dsadatabase.db
81 rows affected.


[]

**Retorna todos os pacientes.**

In [30]:
%%sql

SELECT * FROM pacientes

 * sqlite:///dsadatabase.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
2,197,70,45,543,30.5,0.158,53,1
8,125,96,0,0,0,0.232,54,1
10,139,80,0,0,27.1,1.441,57,0
1,189,60,23,846,30.1,0.398,59,1
5,166,72,19,175,25.8,0.587,51,1
11,143,94,33,146,36.6,0.254,51,1
13,145,82,19,110,22.2,0.245,57,0
5,109,75,26,0,36,0.546,60,0
4,111,72,47,207,37.1,1.39,56,1
9,171,110,24,240,45.4,0.721,54,1


**Vamos alterar a tabela e incluir uma nova coluna.**

In [31]:
%%sql 

ALTER TABLE pacientes
ADD Perfil VARCHAR(10);

 * sqlite:///dsadatabase.db
Done.


[]

**Coluna criada:**

In [32]:
%%sql

SELECT * FROM pacientes

 * sqlite:///dsadatabase.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Perfil
2,197,70,45,543,30.5,0.158,53,1,None
8,125,96,0,0,0,0.232,54,1,None
10,139,80,0,0,27.1,1.441,57,0,None
1,189,60,23,846,30.1,0.398,59,1,None
5,166,72,19,175,25.8,0.587,51,1,None
11,143,94,33,146,36.6,0.254,51,1,None
13,145,82,19,110,22.2,0.245,57,0,None
5,109,75,26,0,36,0.546,60,0,None
4,111,72,47,207,37.1,1.39,56,1,None
9,171,110,24,240,45.4,0.721,54,1,None


**Agora vamos atualizar a coluna com o valor "Normal" se o BMI for menor do que 30.**

In [33]:
%%sql

UPDATE pacientes
SET Perfil = 'Normal'
WHERE BMI < 30;

 * sqlite:///dsadatabase.db
38 rows affected.


[]

**Conferimos o resultado:**

In [34]:
%%sql

SELECT * FROM pacientes

 * sqlite:///dsadatabase.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Perfil
2,197,70,45,543,30.5,0.158,53,1,None
8,125,96,0,0,0,0.232,54,1,Normal
10,139,80,0,0,27.1,1.441,57,0,Normal
1,189,60,23,846,30.1,0.398,59,1,None
5,166,72,19,175,25.8,0.587,51,1,Normal
11,143,94,33,146,36.6,0.254,51,1,None
13,145,82,19,110,22.2,0.245,57,0,Normal
5,109,75,26,0,36,0.546,60,0,None
4,111,72,47,207,37.1,1.39,56,1,None
9,171,110,24,240,45.4,0.721,54,1,None


**Agora vamos atualizar a coluna com o valor "Obeso" se o BMI for maior ou igual a 30.**

In [35]:
%%sql

UPDATE pacientes
SET Perfil = 'Obeso'
WHERE BMI >= 30;

 * sqlite:///dsadatabase.db
43 rows affected.


[]

**Conferimos o resultado:**

In [36]:
%%sql

SELECT * FROM pacientes

 * sqlite:///dsadatabase.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Perfil
2,197,70,45,543,30.5,0.158,53,1,Obeso
8,125,96,0,0,0,0.232,54,1,Normal
10,139,80,0,0,27.1,1.441,57,0,Normal
1,189,60,23,846,30.1,0.398,59,1,Obeso
5,166,72,19,175,25.8,0.587,51,1,Normal
11,143,94,33,146,36.6,0.254,51,1,Obeso
13,145,82,19,110,22.2,0.245,57,0,Normal
5,109,75,26,0,36,0.546,60,0,Obeso
4,111,72,47,207,37.1,1.39,56,1,Obeso
9,171,110,24,240,45.4,0.721,54,1,Obeso


## Retornando os Dados Para o Pandas e Salvando o CSV

In [37]:
# Query
dsa_query = cnn.execute("SELECT * FROM pacientes")

In [38]:
dsa_query

In [39]:
# List Comprehension para retornar os metadados da tabela (nomes de colunas)
cols = [coluna[0] for coluna in dsa_query.description]

In [40]:
cols

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome',
 'Perfil']

In [41]:
# Gera o dataframe
resultado = pd.DataFrame.from_records(data = dsa_query.fetchall(), columns = cols)

In [42]:
# Shape
resultado.shape

(81, 10)

In [43]:
# Visualiza
resultado.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Perfil
0,2,197,70,45,543,30.5,0.158,53,1,Obeso
1,8,125,96,0,0,0.0,0.232,54,1,Normal
2,10,139,80,0,0,27.1,1.441,57,0,Normal
3,1,189,60,23,846,30.1,0.398,59,1,Obeso
4,5,166,72,19,175,25.8,0.587,51,1,Normal


In [44]:
# Salva em CSV
resultado.to_csv('resultado.csv', index = False)

In [45]:
%reload_ext watermark
%watermark -a "Clênio Moura"

Author: Clênio Moura



In [46]:
%watermark

Last updated: 2024-07-24T05:47:06.029998-03:00

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.20.0

Compiler    : GCC 11.2.0
OS          : Linux
Release     : 6.5.0-44-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit



In [47]:
%watermark --iversions

sqlite3: 2.6.0
pandas : 2.1.4



# Fim